### **FEATURE ENGINEERING FOR AIRBNB SEASONALITY ANALYSIS**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

LOAD DATA AND SETUP

In [ ]:
df = pd.read_csv('')

DATE COMPONENTS

In [3]:
df['last_review'] = pd.to_datetime(df['last_review'],errors='coerce')

In [4]:
if pd.api.types.is_datetime64_any_dtype(df['last_review']):
    print(" last_review is datetime")
    print(f"From: {df['last_review'].min().date()}")
    print(f"To: {df['last_review'].max().date()}")
else:
    print("last_review is NOT datetime")

 last_review is datetime
From: 2012-08-25
To: 2025-06-26


In [5]:
# RANGE OF KEY COLUMNS

key_cols = ['price','minimum_nights','availability_365','reviews_per_month']
for col in key_cols:
    if col in df.columns:
        print(f"- {col}: {df[col].min():.2f} to {df[col].max():.2f}")

- price: 50.00 to 1200.00
- minimum_nights: 1.00 to 365.00
- availability_365: 0.00 to 365.00
- reviews_per_month: 0.00 to 25.23


In [6]:
print(f"\n📊 SAMPLE DATA:")
df[['name', 'last_review', 'price', 'reviews_per_month', 'neighbourhood_group']].head()


📊 SAMPLE DATA:


,name,last_review,price,reviews_per_month,neighbourhood_group
0,Clean & quiet apt home by the park,2021-10-19,966.0,0.21,Brooklyn
1,Skylit Midtown Castle,2022-05-21,142.0,0.38,Manhattan
2,Entire Apt: Spacious Studio/Loft by central park,2018-11-19,204.0,0.10,Manhattan
3,Large Furnished Room Near B'way,2019-06-24,1018.0,3.47,Manhattan
4,Beautiful 1br on Upper West Side,2019-06-22,606.0,0.43,Manhattan


In [7]:
print(F"ALL COLUMNS ({len(df.columns)}): ")
for i, col in enumerate(df.columns,1):
    print(f" {i:2d}.{col}")

ALL COLUMNS (27): 
  1.id
  2.name
  3.host_id
  4.host_identity_verified
  5.host_name
  6.neighbourhood_group
  7.neighbourhood
  8.lat
  9.long
 10.country
 11.country_code
 12.instant_bookable
 13.cancellation_policy
 14.room_type
 15.construction_year
 16.price
 17.service_fee
 18.minimum_nights
 19.number_of_reviews
 20.last_review
 21.reviews_per_month
 22.review_rate_number
 23.calculated_host_listings_count
 24.availability_365
 25.house_rules
 26.license
 27.missing_location


In [8]:
print(f"\n✨ KEY COLUMNS FOR SEASONALITY ANALYSIS: ")

# define important categories

columns_importance = {
    'Critical': [
        'last_review',              # Time series base
        'reviews_per_month',        # Demand 
        'price'                     # Revenue
    ],
    'High Priority': [
        'number_of_reviews',        # Total Demand
        'availability_365',         # Supply metric
        'minimum_nights',           # Booking patterns
        'neighbourhood_group',      # Location segmentation
        'room_type'                 # Property segmentation
    ],
    'Medium Priority': [
        'service_fee',              # Additional pricing
        'calculated_host_listings_count',   # Host type indicator
        'lat','long'                # Geospatial analysis
    ]
}

for category, cols in columns_importance.items():
    print(f"\n{category}:")
    for col in cols:
        if col in df.columns:
            print(f"{col} found")
        else:
            print(f"{col} not available")


✨ KEY COLUMNS FOR SEASONALITY ANALYSIS: 

Critical:
last_review found
reviews_per_month found
price found

High Priority:
number_of_reviews found
availability_365 found
minimum_nights found
neighbourhood_group found
room_type found

Medium Priority:
service_fee found
calculated_host_listings_count found
lat found
long found


In [9]:
# Analysis columns

analysis_columns =[]
for category, cols in columns_importance.items():
    analysis_columns.extend([col for col in cols if col in df.columns])
print(f"\nTotal columns for analysis: {len(analysis_columns)}")
print(f"    {analysis_columns}")


Total columns for analysis: 12
    ['last_review', 'reviews_per_month', 'price', 'number_of_reviews', 'availability_365', 'minimum_nights', 'neighbourhood_group', 'room_type', 'service_fee', 'calculated_host_listings_count', 'lat', 'long']


##### **TIME-BASED FEATURES**

In [10]:
print("=" * 30)
print("EXTRACTING DATE FEATURES")
print("=" * 30)

# Extracting date components from last_review
df['year'] = df['last_review'].dt.year
df['month'] = df['last_review'].dt.month
df['day'] = df['last_review'].dt.day
df['day_of_week'] = df['last_review'].dt.day_of_week
df['day_name'] = df['last_review'].dt.day_name()
df['quarter'] = df['last_review'].dt.quarter
df['week_of_year'] = df['last_review'].dt.isocalendar().week

# Sample data
df[['last_review','year','month','quarter','day_name']].head(10)

EXTRACTING DATE FEATURES


,last_review,year,month,quarter,day_name
0,2021-10-19,2021,10,4,Tuesday
1,2022-05-21,2022,5,2,Saturday
2,2018-11-19,2018,11,4,Monday
3,2019-06-24,2019,6,2,Monday
4,2019-06-22,2019,6,2,Saturday
5,2019-06-23,2019,6,2,Sunday
6,2019-06-24,2019,6,2,Monday
7,2018-10-31,2018,10,4,Wednesday
8,2019-06-29,2019,6,2,Saturday
9,2019-06-28,2019,6,2,Friday


##### **SEASONALITY INDICATORS**

In [11]:
print("=" * 30)
print("CREATING SEASON CATEGORIES")
print("=" * 30)

def assign_season(month):
    if month in [12,1,2]:
        return 'Winter'
    elif month in [3,4,5]:
        return 'Summer'
    elif month in [6,7,8]:
        return 'Summer'
    else :  # month in 9.10,11
        return 'Fall'

df['season'] = df['month'].apply(assign_season)

print("season categories created: ")
print(df['season'].value_counts().sort_index())
print('\n')
print("SAMPLE WITH SEASONS: ")
df[['last_review','month','season','reviews_per_month']].head(10)

CREATING SEASON CATEGORIES
season categories created: 
season
Fall       6055
Summer    32647
Winter    11828
Name: count, dtype: int64


SAMPLE WITH SEASONS: 


,last_review,month,season,reviews_per_month
0,2021-10-19,10,Fall,0.21
1,2022-05-21,5,Summer,0.38
2,2018-11-19,11,Fall,0.10
3,2019-06-24,6,Summer,3.47
4,2019-06-22,6,Summer,0.43
5,2019-06-23,6,Summer,1.50
6,2019-06-24,6,Summer,1.34
7,2018-10-31,10,Fall,0.22
8,2019-06-29,6,Summer,1.20
9,2019-06-28,6,Summer,1.72


In [12]:
def is_peak_season(month):
    peak_months =[6,7,8,12]
    return 1 if month in peak_months else 0

df['is_peak_season'] = df['month'].apply(is_peak_season)

df[['month','season','is_peak_season']].head(10)

,month,season,is_peak_season
0,10,Fall,0
1,5,Summer,0
2,11,Fall,0
3,6,Summer,1
4,6,Summer,1
5,6,Summer,1
6,6,Summer,1
7,10,Fall,0
8,6,Summer,1
9,6,Summer,1


CREATING WEEKEND / WEEKDAY FLAG

In [13]:
print("="*30)
print("CREATING WEEKEND/WEEKDAY FLAGS")
print("="*30)

df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
print("weekend flag created")
print(f"Weekday Reviews: {(df['is_weekend']==0).sum():,}")
print(f"Weekend Reviews: {(df['is_weekend']==1).sum():,}")

CREATING WEEKEND/WEEKDAY FLAGS
weekend flag created
Weekday Reviews: 29,555
Weekend Reviews: 20,975


In [14]:
print("\nDISTRIBUTION")
weekend_distribution = df.groupby('is_weekend')['reviews_per_month'].mean()
print(f"-> Average reviews/month on weekdays: {weekend_distribution[0]:.2f}")
print(f"-> Average reviews/month on weekends: {weekend_distribution[1]:.2f}")


DISTRIBUTION
-> Average reviews/month on weekdays: 1.33
-> Average reviews/month on weekends: 1.49


In [15]:
print("="*30)
print("SEASONALITY SUMMARY")
print("="*30)

new_features=['year','month','day','quarter','day_of_week','day_name',
              'week_of_year','season','is_weekend','is_peak_season']

print(f"\nNEW FEATURES CREATED: {len(new_features)}")
for i, feature in enumerate(new_features,1):
    print(f"{i}. {feature}")

print(f"\n UPDATED DATASET: ")
print(f" -> Total columns: {len(df.columns)}")
print(f" -> New Date Features: {len(new_features)}")

# All columns with new features
print("\n SAMPLE WITH NEW FEATURES: ")
display_cols = ['last_review', 'year', 'month', 'quarter', 'season', 
                'day_name', 'is_weekend', 'price', 'reviews_per_month','is_peak_season']
df[display_cols].head()

SEASONALITY SUMMARY

NEW FEATURES CREATED: 10
1. year
2. month
3. day
4. quarter
5. day_of_week
6. day_name
7. week_of_year
8. season
9. is_weekend
10. is_peak_season

 UPDATED DATASET: 
 -> Total columns: 37
 -> New Date Features: 10

 SAMPLE WITH NEW FEATURES: 


,last_review,year,month,quarter,season,day_name,is_weekend,price,reviews_per_month,is_peak_season
0,2021-10-19,2021,10,4,Fall,Tuesday,0,966.0,0.21,0
1,2022-05-21,2022,5,2,Summer,Saturday,1,142.0,0.38,0
2,2018-11-19,2018,11,4,Fall,Monday,0,204.0,0.10,0
3,2019-06-24,2019,6,2,Summer,Monday,0,1018.0,3.47,1
4,2019-06-22,2019,6,2,Summer,Saturday,1,606.0,0.43,1


##### **DEMAND INDICATORS**

In [16]:
print("="*30)
print("DEMAND INDICATORS")
print("="*30)

print("\nREVIEWS PER MONTH DISTRIBUTION:")
df['reviews_per_month'].describe()

DEMAND INDICATORS

REVIEWS PER MONTH DISTRIBUTION:


count    50530.000000
mean         1.395898
std          1.563011
min          0.000000
25%          0.240000
50%          0.795000
75%          2.120000
max         25.230000
Name: reviews_per_month, dtype: float64

In [17]:
# Creating categories based on quartiles

df['review_frequency_category'] = pd.qcut(
    df['reviews_per_month'],
    q=4,
    labels =['Low','Medium','High','Very High'],
    duplicates='drop'
)

print("\nReview Frequency categories created: ")
df['review_frequency_category'].value_counts().sort_index()


Review Frequency categories created: 


review_frequency_category
Low          12864
Medium       12401
High         12669
Very High    12596
Name: count, dtype: int64

In [18]:
# Average price for frequencies

print("\nAverage price by review frequency: ")
print(df.groupby('review_frequency_category')['price'].mean().round(2))


Average price by review frequency: 
review_frequency_category
Low          628.07
Medium       628.67
High         621.56
Very High    627.90
Name: price, dtype: float64


In [19]:
print("="*30)
print("AVAILABILITY ANALYSIS")
print("="*30)

print("\nAVAILABILITY_365 DISTRIBUTION:")
df['availability_365'].describe()

print("\nREVIEWS_PER_MONTH PERCENTILES:")
percentiles = [0, 25, 50, 75, 90, 95, 99, 100]
for p in percentiles:
    value = df['reviews_per_month'].quantile(p/100)
    print(f"   {p}th percentile: {value:.2f}")



AVAILABILITY ANALYSIS

AVAILABILITY_365 DISTRIBUTION:

REVIEWS_PER_MONTH PERCENTILES:
   0th percentile: 0.00
   25th percentile: 0.24
   50th percentile: 0.80
   75th percentile: 2.12
   90th percentile: 3.56
   95th percentile: 4.49
   99th percentile: 6.66
   100th percentile: 25.23


In [20]:
df['demand_score'] = pd.qcut(
    df['reviews_per_month'],
    q=5,
    labels=[20, 40, 60, 80, 100],
    duplicates='drop'
).astype(float)

print("\nDemand Score Recalculated (Percentile-based):")
print(f"   • Mean: {df['demand_score'].mean():.2f}")
print(f"   • Median: {df['demand_score'].median():.2f}")
print(f"   • Distribution:")
print(df['demand_score'].value_counts().sort_index())

# Recreate demand level categories
df['demand_level'] = pd.cut(
    df['demand_score'],
    bins=[0, 25, 50, 75, 100],
    labels=['Low', 'Medium', 'High', 'Very High'],
    include_lowest=True
)


Demand Score Recalculated (Percentile-based):
   • Mean: 59.79
   • Median: 60.00
   • Distribution:
demand_score
20.0     10411
40.0      9898
60.0     10107
80.0     10026
100.0    10088
Name: count, dtype: int64


In [21]:
print("\nNEW DEMAND LEVEL DISTRIBUTION:")
print(df['demand_level'].value_counts().sort_index())

print("\nSAMPLE - CORRECTED DEMAND INDICATORS:")
print(df[['reviews_per_month', 'demand_score', 'demand_level']].head(15))

print("\nINTERPRETATION:")
print("   • Low: 0 reviews/month to 25th percentile")
print("   • Medium: 25th to 50th percentile")
print("   • High: 50th to 75th percentile")
print("   • Very High: 75th percentile and above")


NEW DEMAND LEVEL DISTRIBUTION:
demand_level
Low          10411
Medium        9898
High         10107
Very High    20114
Name: count, dtype: int64

SAMPLE - CORRECTED DEMAND INDICATORS:
    reviews_per_month  demand_score demand_level
0                0.21          40.0       Medium
1                0.38          40.0       Medium
2                0.10          20.0          Low
3                3.47         100.0    Very High
4                0.43          40.0       Medium
5                1.50          80.0    Very High
6                1.34          80.0    Very High
7                0.22          40.0       Medium
8                1.20          80.0    Very High
9                1.72          80.0    Very High
10               4.44         100.0    Very High
11               0.07          20.0          Low
12               1.37          80.0    Very High
13               0.49          40.0       Medium
14               1.11          60.0         High

INTERPRETATION:
   • Low: 0 r

##### **PRICE CATEGORIES**

In [22]:
print("="*30)
print("PRICE CATEGORIES")
print("="*30)

print("\nPRICE DISTRIBUTION: ")
df['price'].describe()

PRICE CATEGORIES

PRICE DISTRIBUTION: 


count    50530.000000
mean       626.541480
std        331.515015
min         50.000000
25%        340.250000
50%        626.000000
75%        914.000000
max       1200.000000
Name: price, dtype: float64

In [23]:
# PRICE OUTLIERS
print("\nPRICE PERCENTILES: ")
percentiles=[10,25,50,75,90,95,99]
for p in percentiles:
    value = df['price'].quantile(p/100)
    print(f" {p}th percentile: ${value:.2f}")


PRICE PERCENTILES: 
 10th percentile: $165.00
 25th percentile: $340.25
 50th percentile: $626.00
 75th percentile: $914.00
 90th percentile: $1084.00
 95th percentile: $1144.00
 99th percentile: $1189.00


In [26]:
print("\nCREATING PRICE CATEGORIES")
print("-"*30)

df['price_tier']=pd.qcut(
    df['price'],
    q=4,
    labels=['Budget','Mid-Range','Upscale','Luxury'],
    duplicates='drop'
)

print("\nPrice Tiers Created: ")
print(df['price_tier'].value_counts().sort_index())


CREATING PRICE CATEGORIES
------------------------------

Price Tiers Created: 
price_tier
Budget       12633
Mid-Range    12639
Upscale      12665
Luxury       12593
Name: count, dtype: int64


In [27]:
print("\nPRICE RANGES BY TIER: ")
price_by_tier=df.groupby('price_tier')['price'].agg(['min','max','mean','count'])
print(price_by_tier)

print("\nAVERAGE REVIEWS BY PRICE TIER: ")
reviews_by_price=df.groupby('price_tier')['reviews_per_month'].mean()
print(reviews_by_price.round(2))


PRICE RANGES BY TIER: 
              min     max         mean  count
price_tier                                   
Budget       50.0   340.0   194.895670  12633
Mid-Range   341.0   626.0   485.099217  12639
Upscale     627.0   914.0   769.870825  12665
Luxury      915.0  1200.0  1057.368459  12593

AVERAGE REVIEWS BY PRICE TIER: 
price_tier
Budget       1.39
Mid-Range    1.41
Upscale      1.37
Luxury       1.41
Name: reviews_per_month, dtype: float64


In [28]:
print("\nVALUE SCORE: Price vs Demand")
print("-"*30)

# value score = how many reviews per dollar spent
df['value_score'] = (df['reviews_per_month']/df['price']*100).round(2)

print("\n Value Score Created (Reviews per $100)")
print(f" -> Mean: {df['value_score'].mean():.2f}")
print(f" -> Median: {df['value_score'].median():.2f}")
print(f" -> Min: {df['value_score'].min():.2f}")
print(f" -> Max: {df['value_score'].max():.2f}")


VALUE SCORE: Price vs Demand
------------------------------

 Value Score Created (Reviews per $100)
 -> Mean: 0.38
 -> Median: 0.15
 -> Min: 0.00
 -> Max: 18.41


In [29]:
# CREATE VALUE CATEGORIES
df['value_category'] = pd.qcut(
    df['value_score'],
    q=3,
    labels= ['Low Value','Good Value','Excellent Value'],
    duplicates= 'drop'
)

print("\nVALUE CATEGORIES DISTRIBUTION: ")
print(df['value_category'].value_counts().sort_index())


VALUE CATEGORIES DISTRIBUTION: 
value_category
Low Value          17836
Good Value         16027
Excellent Value    16667
Name: count, dtype: int64


In [30]:
print("\nINTERPRETATION: ")
df[['price','reviews_per_month','price_tier','value_score','value_category']].head(10)


INTERPRETATION: 


,price,reviews_per_month,price_tier,value_score,value_category
0,966.0,0.21,Luxury,0.02,Low Value
1,142.0,0.38,Budget,0.27,Good Value
2,204.0,0.10,Budget,0.05,Low Value
3,1018.0,3.47,Luxury,0.34,Excellent Value
4,606.0,0.43,Mid-Range,0.07,Low Value
5,714.0,1.50,Upscale,0.21,Good Value
6,580.0,1.34,Mid-Range,0.23,Good Value
7,578.0,0.22,Mid-Range,0.04,Low Value
8,778.0,1.20,Upscale,0.15,Good Value
9,656.0,1.72,Upscale,0.26,Good Value


##### **BOOKING PATTERNS**

In [36]:
print("\nBOOKING PATTERNS")
print("-"*20)

print("\nMINIMUM NIGHTS DISTRIBUTIONS: ")
print(df['minimum_nights'].describe())

print("\nMINIMUM NIGHTS PERCENTILES: ")
percentiles=[10,25,50,75,90,95]
for p in percentiles:
    value=df['minimum_nights'].quantile(p/100)
    print(f" {p}th percentile: {value:.0f} nights")


BOOKING PATTERNS
--------------------

MINIMUM NIGHTS DISTRIBUTIONS: 
count    50530.000000
mean         7.214031
std         15.508975
min          1.000000
25%          2.000000
50%          3.000000
75%          5.000000
max        365.000000
Name: minimum_nights, dtype: float64

MINIMUM NIGHTS PERCENTILES: 
 10th percentile: 1 nights
 25th percentile: 2 nights
 50th percentile: 3 nights
 75th percentile: 5 nights
 90th percentile: 30 nights
 95th percentile: 30 nights


In [37]:
print("CREATING STAY LENGTH CATEGORIES")
print("-"*30)

def categorize_stay(nights):
    if nights <= 1:
        return 'Nightly'
    elif nights <= 7:
        return 'Weekly'
    elif nights <=30:
        return 'Monthly'
    else:
        return 'Long-term'
    
df['stay_length_category'] = df['minimum_nights'].apply(categorize_stay)
print("Stay length categories created: ")
print(df['stay_length_category'].value_counts())

CREATING STAY LENGTH CATEGORIES
------------------------------
Stay length categories created: 
stay_length_category
Weekly       29643
Nightly      12549
Monthly       7664
Long-term      674
Name: count, dtype: int64


In [38]:
print("\nREVIEWS BY STAY TYPE: ")
stay_reviews= df.groupby('stay_length_category')['reviews_per_month'].agg(['mean','median','count'])
print(stay_reviews.round(2))


REVIEWS BY STAY TYPE: 
                      mean  median  count
stay_length_category                     
Long-term             0.56    0.27    674
Monthly               0.53    0.29   7664
Nightly               2.10    1.45  12549
Weekly                1.34    0.91  29643


In [39]:
print("\nINTERPRETATION:")
print("   • Nightly: Most flexible - attracts tourists/travelers")
print("   • Weekly: Standard short-term rentals")
print("   • Monthly: Targets longer stays - less turnover")
print("   • Long-term: Corporate/relocation - stable income")


INTERPRETATION:
   • Nightly: Most flexible - attracts tourists/travelers
   • Weekly: Standard short-term rentals
   • Monthly: Targets longer stays - less turnover
   • Long-term: Corporate/relocation - stable income


In [41]:
print("BOOKING FLEXIBILITY ANALYSIS")
print("-" * 30)

# create flexibility score: shorter minimum nights = more flexible=10, longer=1
df['flexibility_score'] = pd.cut(
    df['minimum_nights'],
    bins=[-1, 1, 7, 30, 365],
    labels=[10, 7, 4, 1],
    include_lowest=True
).astype(float)

print("\nFLEXIBILITY SCORE CREATED (1-10 SCALE): ")
print(f" -> 10 = Very Flexible (nightly)")
print(f" -> 7 = Flexible (weekly)")
print(f" -> 4 = Moderate (monthly)")
print(f" -> 1 = Rigid (30+ days)")

BOOKING FLEXIBILITY ANALYSIS
------------------------------

FLEXIBILITY SCORE CREATED (1-10 SCALE): 
 -> 10 = Very Flexible (nightly)
 -> 7 = Flexible (weekly)
 -> 4 = Moderate (monthly)
 -> 1 = Rigid (30+ days)


In [44]:
print("\nDISTRIBUTION: ")
print(df['flexibility_score'].value_counts().sort_index(ascending=False))

print("\nDEMAND BY FLEXIBILITY: ")
flex_demand=df.groupby('flexibility_score')['reviews_per_month'].mean()
print(flex_demand.round(3))

print("\nKEY FINDING: ")
if flex_demand.iloc[0] > flex_demand.iloc[-1]:
    print(f" -> More flexible listings get MORE bookings ({flex_demand.iloc[0]:.2f} vs {flex_demand.iloc[-1]:.2f} reviews/month)")
else:
    print(f" -> Minimum stay length has similar impact on all listings")


DISTRIBUTION: 
flexibility_score
10.0    12549
7.0     29643
4.0      7664
1.0       674
Name: count, dtype: int64

DEMAND BY FLEXIBILITY: 
flexibility_score
1.0     0.555
4.0     0.533
7.0     1.340
10.0    2.100
Name: reviews_per_month, dtype: float64

KEY FINDING: 
 -> Minimum stay length has similar impact on all listings


In [49]:
print("SEASONALITY BY BOOKING TYPE")
print("-"*30)

seasonality_by_type = df.groupby(['season','stay_length_category'])['reviews_per_month'].agg(['mean','count']).round(2)
print("DEMAND BY SEASON AND STAY TYPE: ")
print(seasonality_by_type)

print("\nSAMPLE - BOOKING PATTERNS: ")
df[['minimum_nights', 'stay_length_category', 'flexibility_score', 
         'season', 'reviews_per_month', 'demand_level']].head(10)

SEASONALITY BY BOOKING TYPE
------------------------------
DEMAND BY SEASON AND STAY TYPE: 
                             mean  count
season stay_length_category             
Fall   Long-term             0.34    108
       Monthly               0.41   1475
       Nightly               0.55   1182
       Weekly                0.37   3290
Summer Long-term             0.64    412
       Monthly               0.57   4047
       Nightly               2.25   8174
       Weekly                1.47  20014
Winter Long-term             0.49    154
       Monthly               0.54   2142
       Nightly               2.28   3193
       Weekly                1.42   6339

SAMPLE - BOOKING PATTERNS: 


,minimum_nights,stay_length_category,flexibility_score,season,reviews_per_month,demand_level
0,10.0,Monthly,4.0,Fall,0.21,Medium
1,30.0,Monthly,4.0,Summer,0.38,Medium
2,10.0,Monthly,4.0,Fall,0.10,Low
3,2.0,Weekly,7.0,Summer,3.47,Very High
4,5.0,Weekly,7.0,Summer,0.43,Medium
5,2.0,Weekly,7.0,Summer,1.50,Very High
6,4.0,Weekly,7.0,Summer,1.34,Very High
7,90.0,Long-term,1.0,Fall,0.22,Medium
8,2.0,Weekly,7.0,Summer,1.20,Very High
9,2.0,Weekly,7.0,Summer,1.72,Very High


##### **VALIDATION & SUMMARY**

In [50]:
print("VALIDATION & SUMMARY")
print("-"*30)

print("\nFeatures created: ")
features_by_category = {
    'Time-Based Features': ['year', 'month', 'quarter', 'day_of_week', 'day_name', 
                            'week_of_year', 'is_weekend', 'is_peak_season'],
    'Seasonality Features': ['season'],
    'Demand Indicators': ['review_frequency_category', 'availability_category', 
                          'demand_score', 'demand_level'],
    'Price Features': ['price_tier', 'value_score', 'value_category'],
    'Booking Patterns': ['stay_length_category', 'flexibility_score']
}

total_features=0
for category,features in features_by_category.items():
    existing_features =[ f for f in features if f in df.columns]
    total_features += len(existing_features)
    print(f"\n{category}: {len(existing_features)}")
    for feature in existing_features:
        print(f" -> {feature}")

print(f"\nTOTAL NEW FEATURES CREATED: {total_features}")
print(f" -> Original columns: 27")
print(f" -> New Columns: {len(df.columns) - 27}")
print(f" -> Total Columns: {len(df.columns)}")

VALIDATION & SUMMARY
------------------------------

Features created: 

Time-Based Features: 8
 -> year
 -> month
 -> quarter
 -> day_of_week
 -> day_name
 -> week_of_year
 -> is_weekend
 -> is_peak_season

Seasonality Features: 1
 -> season

Demand Indicators: 3
 -> review_frequency_category
 -> demand_score
 -> demand_level

Price Features: 3
 -> price_tier
 -> value_score
 -> value_category

Booking Patterns: 2
 -> stay_length_category
 -> flexibility_score

TOTAL NEW FEATURES CREATED: 17
 -> Original columns: 27
 -> New Columns: 18
 -> Total Columns: 45


DATA QUALITY CHECK

In [51]:
print(f"Rows: {len(df):,}")
print(f"Columns: {len(df.columns)}")

print("\nMISSING VALUES CHECK: ")
missing_cols = df.isnull().sum()
missing_cols = missing_cols[missing_cols > 0]

if len(missing_cols) ==0:
    print("NO MISSING VALUES FOUND")
else:
    print("Found missing values: ")
    for col , count in missing_cols.items():
        print(f" -> {col}: {count}")

print("\nDUPLICATED CHECK: ")
dupes=df.duplicated().sum()
if dupes==0:
    print("no duplicate rows found")
else:
    print(f" -> {dupes} duplicate rows")


print("\nDATA TYPE CHECK: ")
dtype_summary=df.dtypes.value_counts()
for dtype, count in dtype_summary.items():
    print(f" -> {dtype}: {count} columns")

Rows: 50,530
Columns: 45

MISSING VALUES CHECK: 
Found missing values: 
 -> lat: 4
 -> long: 4

DUPLICATED CHECK: 
no duplicate rows found

DATA TYPE CHECK: 
 -> object: 14 columns
 -> float64: 14 columns
 -> int32: 5 columns
 -> int64: 4 columns
 -> bool: 2 columns
 -> category: 2 columns
 -> datetime64[ns]: 1 columns
 -> UInt32: 1 columns
 -> category: 1 columns
 -> category: 1 columns


FEATURES DISTRIBUTION SUMMARY

In [56]:
print("\nTIME FEATURES: ")
print(f" -> Years covered: {df['year'].nunique()} ({df['year'].min()} - {df['year'].max()})")
print(f" -> Months: {df['month'].unique()}")
print(f" -> Seasons: {df['season'].unique()}")
print(f" -> Day distribution:\n{df['day_name'].value_counts().sort_index()}")


TIME FEATURES: 
 -> Years covered: 12 (2012 - 2025)
 -> Months: [10  5 11  6 12  8  4  3  1  9  2  7]
 -> Seasons: ['Fall' 'Summer' 'Winter']
 -> Day distribution:
day_name
Friday        5985
Monday        8910
Saturday      7165
Sunday       13810
Thursday      4794
Tuesday       5217
Wednesday     4649
Name: count, dtype: int64


In [57]:
print("\nDEMAND LEVELS: ")
print(df['demand_level'].value_counts().sort_index())


DEMAND LEVELS: 
demand_level
Low          10411
Medium        9898
High         10107
Very High    20114
Name: count, dtype: int64


In [58]:
print("\nPRICE TIERS: ")
print(df['price_tier'].value_counts().sort_index())


PRICE TIERS: 
price_tier
Budget       12633
Mid-Range    12639
Upscale      12665
Luxury       12593
Name: count, dtype: int64


In [59]:
print("\nBOOKING TYPES: ")
print(df['stay_length_category'].value_counts())


BOOKING TYPES: 
stay_length_category
Weekly       29643
Nightly      12549
Monthly       7664
Long-term      674
Name: count, dtype: int64


In [60]:
print("\nVALUE CATEGORIES:")
print(df['value_category'].value_counts().sort_index())



VALUE CATEGORIES:
value_category
Low Value          17836
Good Value         16027
Excellent Value    16667
Name: count, dtype: int64


CORRELATION ANALYSIS PREVIEW

In [65]:
# SELECT NUMERIC COLUMNS FOR CORRELATION
numeric_cols_for_corr = ['price', 'minimum_nights', 'availability_365', 
                         'reviews_per_month', 'demand_score', 'flexibility_score', 
                         'value_score']

# CREATE CORRELATION MATRIX
corr_with_demand = df[numeric_cols_for_corr].corr()['reviews_per_month'].sort_values(ascending=False)

print("\nCORRELATION WITH REVIEWS_PER_MONTH (Demand Proxy): ")
for feature, corr in corr_with_demand.items():
    if feature != 'reviews_per_month':
        strength = 'Strong' if abs(corr) > 0.3 else 'Moderate' if abs(corr) > 0.1 else 'Weak'
        direction = '⬆️' if corr>0 else '⬇️'
        print(f" {direction} {feature}: {corr:.3f} ({strength})")


CORRELATION WITH REVIEWS_PER_MONTH (Demand Proxy): 
 ⬆️ demand_score: 0.834 (Strong)
 ⬆️ value_score: 0.562 (Strong)
 ⬆️ flexibility_score: 0.314 (Strong)
 ⬆️ availability_365: 0.104 (Moderate)
 ⬆️ price: 0.004 (Weak)
 ⬇️ minimum_nights: -0.190 (Moderate)


1. demand_score - (strong)  
2. value_score - (strong) high demand
3. flexibility_score - (strong) flexible stays = more bookings
4. availability_365 - (moderate) more available = slightly more reviews
5. price - (none) price doesnt matter
6.  minimum_nights - (moderated negetive) Longer stays = fewer bookings

In [66]:
print("\nINTERPRETATION:")
print("   ↑ = Positive correlation (increases demand)")
print("   ↓ = Negative correlation (decreases demand)")
print("   • Strong (>0.3): Major influence")
print("   • Moderate (0.1-0.3): Some influence")
print("   • Weak (<0.1): Little influence")


INTERPRETATION:
   ↑ = Positive correlation (increases demand)
   ↓ = Negative correlation (decreases demand)
   • Strong (>0.3): Major influence
   • Moderate (0.1-0.3): Some influence
   • Weak (<0.1): Little influence


##### **SUMMARY**

In [68]:
print("\n" + "=" * 80)
print("EATURE ENGINEERING COMPLETE!")
print("=" * 80)

print(f"""
SUMMARY OF ENGINEERED FEATURES:

-> Time-Based Features (8):
   - Extracted from last_review: year, month, quarter, day_of_week
   - Day names and week numbers
   - Peak season flags

-> Seasonality Indicators (1):
   - Season categories (Winter/Spring/Summer/Fall)

-> Demand Indicators (4):
   - Review frequency categories (Low/Medium/High/Very High)
   - Availability categories 
   - Demand score (0-100)
   - Demand level

-> Price Features (3):
   - Price tiers (Budget/Mid-Range/Upscale/Luxury)
   - Value score (reviews per $100)
   - Value categories

-> Booking Pattern Features (2):
   - Stay length categories (Nightly/Weekly/Monthly/Long-term)
   - Flexibility score (1-10)

-> DATASET READY FOR:
   ✓ Exploratory Data Analysis (EDA)
   ✓ Seasonality visualization
   ✓ Time series decomposition
   ✓ Demand pattern analysis
   ✓ Predictive modeling (if needed)
""")

print("=" * 80)



EATURE ENGINEERING COMPLETE!

SUMMARY OF ENGINEERED FEATURES:

-> Time-Based Features (8):
   - Extracted from last_review: year, month, quarter, day_of_week
   - Day names and week numbers
   - Peak season flags

-> Seasonality Indicators (1):
   - Season categories (Winter/Spring/Summer/Fall)

-> Demand Indicators (4):
   - Review frequency categories (Low/Medium/High/Very High)
   - Availability categories 
   - Demand score (0-100)
   - Demand level

-> Price Features (3):
   - Price tiers (Budget/Mid-Range/Upscale/Luxury)
   - Value score (reviews per $100)
   - Value categories

-> Booking Pattern Features (2):
   - Stay length categories (Nightly/Weekly/Monthly/Long-term)
   - Flexibility score (1-10)

-> DATASET READY FOR:
   ✓ Exploratory Data Analysis (EDA)
   ✓ Seasonality visualization
   ✓ Time series decomposition
   ✓ Demand pattern analysis
   ✓ Predictive modeling (if needed)



SAVE THE DATASET

In [69]:
output_path = '/Users/starboy/Documents/Projects/Airbnb/engineered_data_for_eda.csv'
df.to_csv(output_path,index=False)